In [47]:
'''
gridworld_terminal_MC.py
'''

# libraries
import numpy as np
import sys
from tqdm import tqdm

# # parameters
# ginDim = 4

GAMMA = 0.9
LEARNING_RATE = 0.1

# state indexing: (x,y) -- for 4x4 grid
# [
#     (0,0), (0,1), (0,2), (0,3)
#     (1,0), (1,1), (1,2), (1,3)
#     (2,0), (2,1), (2,2), (2,3)
#     (3,0), (3,1), (3,2), (3,3)
# ]

# left (0), up (1), right (2), down (3)
v_actions = [np.array([0, -1]),
            np.array([-1, 0]),
            np.array([0, 1]),
            np.array([1, 0])]

i_nActions = len(v_actions)


def is_terminal(state, ginDim):
    x, y = state
    return (x == 0 and y == 0) or (x == ginDim - 1 and y == ginDim - 1)

def get_initial_state(ginDim):
    # print("ginDim:%s" %ginDim)
    while True:
        state_i = np.random.choice(ginDim)
        state_j = np.random.choice(ginDim)
        if (state_i == 0 and state_j == 0) or (state_i == ginDim-1 and state_j == ginDim-1):
            continue
        else:
            break
    return state_i, state_j

def target_policy_agent(state, gvPolicy, action=None):
    # print(state, action)
    if action is None:
        # t_iActionID = np.random.choice(i_nActions)
        t_iActionID = np.random.choice(list(range(i_nActions)), p=gvPolicy)
        # return the selected action and the prob of selection
        return t_iActionID, v_actions[t_iActionID], gvPolicy[t_iActionID]
    else:
        # return the given action and the prob of selecting that action at given state
        return action, v_actions[action], gvPolicy[action]


def step(giSt, giAc, ginDim):
    giSt = np.array(giSt)
    next_state = (giSt + giAc).tolist()
    x, y = next_state
    reward = -1

    # if you exit the grid, return the original state
    if x < 0 or x >= ginDim or y < 0 or y >= ginDim:
        next_state = giSt.tolist()

    return next_state, reward


def mc_travel(ginDim, policy_agent, gvPolicy, initial_state=None, initial_action=None):

    # trajectory of agent
    agent_trajectory = []

    # cumulative_reward
    total_reward = 0

    if initial_state is None:
        # generate a random initial state
        state_i, state_j = get_initial_state(ginDim)
    else:
        state_i, state_j = initial_state
    
    # initial state
    state = (state_i, state_j)

    if initial_action is None:
        initial_action = np.random.choice(list(range(i_nActions)))

    #initial action    
    action = initial_action

    '''
    complete implementation for the rest of the episode
    starting with below while loop
    '''
    states_actions_rewards = [(state,action,0)] # list of tuples of (state,reward)
    
    while is_terminal(state, ginDim) == False:
         # get new action
         i_action, action, prob = policy_agent(state, gvPolicy)
         state, reward = step(state, action, ginDim)
         states_actions_rewards.append((state,i_action,reward))
    
    #calculate returns by working backwards from terminal state
    G = 0
    first=True
   
    for s,a,r in reversed(states_actions_rewards):
        #the value of the terminal state is 0 by definition
        #we should ignore the first state we encounter
        if first:
            first = False
        else:
            agent_trajectory.append((s,a,G))

        total_reward = G
        G = r + GAMMA*G

    agent_trajectory.reverse() # we want it to be in order of state visited

    return total_reward, agent_trajectory


def monte_carlo_on_policy_prediction(gvPolicy, episodes=1000, ginDim=4, first_visit_MC=True):

    '''
    gvPolicy (i.e. provided policy) is assumed to be same for each state
    it only describes a probability of taking each action at a given state
    gvPolicy is expected to be in the following form:
    gvPolicy = [1/4, 1/4, 1/4, 1/4]
    '''
    
    # initialize
    state_values = np.zeros((ginDim, ginDim))
    state_counts = np.zeros((ginDim, ginDim))

    # to prevent zero division error
    state_counts[(0,0)] = 1
    state_counts[(ginDim-1,ginDim-1)] = 1

    returns = {} #dictionary of state -> list of returns we reveived
    for i in range(ginDim):
        for j in range(ginDim):
            returns[(i,j)] = []

    # travel for several episodes
    for episode in tqdm(range(0, episodes)):
    # for i in range(0, episodes):

        # play the game based on target_policy_player
        reward, agent_trajectory = mc_travel(ginDim, target_policy_agent, gvPolicy)
        '''
        complete implementation for the rest of the episode
        that involves implementing 'first visit check' or 'every visit MC' to record state_values and state_counts for the episode
        '''
        seen_states = set()
        for s,a,G in agent_trajectory:
            x,y = s
            update = False

            if first_visit_MC:
              if (x,y) not in seen_states:
                seen_states.add((x,y))
                update = True
            
            else: update = True
            
            if update :
                returns[(x,y)].append(G)
                state_values[s] = np.sum(returns[(x,y)])
                state_counts[s] +=1


    return episodes, np.around(state_values/state_counts,1)


def max_dict(d):
    '''
    :param d: dictionary
    :return: returns argmax (key) and max (value)
    '''
    max_key = None
    max_val = float('-inf')
    for k,v in d.items():
        if v > max_val:
            max_val = v
            max_key = k
    return max_key, max_val


def monte_carlo_ES(gvPolicy, episodes=1000, ginDim=4, first_visit_MC=True):
    '''
    gvPolicy is used for target policy in an MC control algorithm
    gvPolicy (i.e. provided policy) is assumed to be same for each state
    it only describes a probability of taking each action at a given state
    gvPolicy is expected to be in the following form:
    gvPolicy = [1/4, 1/4, 1/4, 1/4]
    '''
    
    # initialize
    state_action_values = np.zeros((ginDim, ginDim, i_nActions))
    state_action_values_avg = np.zeros((ginDim, ginDim, i_nActions))
    # to avoid zero division error
    state_action_pair_count = np.ones((ginDim, ginDim, i_nActions))
    best_actions = np.zeros((ginDim, ginDim))

    #state -> action
    #initialize a random policy
    policy = {}
    for i in range(ginDim):
        for j in range(ginDim):
            policy[(i,j)] = np.random.choice(list(range(i_nActions)))
    
    #initialize Q(s,a) and returns
    Q = {}
    returns = {} # dictionary of state:list of returns recieved

    for i in range(ginDim):
        for j in range(ginDim):
            #if is_terminal((i,j), ginDim) == False: # not terminal
                Q[(i,j)] = {}
                for i_action, action in enumerate(v_actions): #ALL_POSSIBLE_ACTIONS
                    Q[(i,j)][i_action] = 0 # needs to be initialized so we can argmax
                    returns[((i,j),i_action)] = []
            #else:
                #terminal state or state we cant get to
            #    pass

    # behaviour policy is greedy
    def behavior_policy(state, gvPolicy, action=None):

        epsilon_pol = 0.1 #0.01
        '''
        implement a behavior_policy agent that generates epsilon-greedy policy
        current return value is a placeholder which only generates a random action
        '''
        p = np.random.random()

        if p < (1-epsilon_pol):
            x,y = state
            action = gvPolicy[(x,y)]

        else:
            action = np.random.choice(list(range(i_nActions)))

        return action, v_actions[action], 1.0
    

    # travel for several episodes
    for episode in tqdm(range(0, episodes)):
        
        # if episode %1000 == 0:
        #     print("episode: %s" %episode)
        
        # for each episode, use a randomly initialized state and action
        initial_state = get_initial_state(ginDim)
        initial_action = np.random.choice(i_nActions)

        current_policy = behavior_policy if episode else target_policy_agent #first execute target, then behaviour
        current_gvPolicy = policy if episode else gvPolicy
        
        # print("initial_state: %s, current_policy(initial_state): %s\n" %(initial_state, current_policy(initial_state)))

        # travel based on current_policy
        reward, agent_trajectory = mc_travel(ginDim, current_policy, current_gvPolicy)
        #print("agent_trajectory:\n%s\n" %agent_trajectory)

        '''
        complete implementation for the rest of the episode
        that involves implementing 'first visit check' or 'every visit MC' to record state_values and state_counts for the episode
        '''
        seen_state_action_pairs = set()

        for s, a, G in agent_trajectory:
            #check if we have already seen
            #called 'first-visit' MC policy evaluation
            x,y = s
            sa = ((x,y),a)
            update = False

            if first_visit_MC:
              if sa not in seen_state_action_pairs:
                seen_state_action_pairs.add(sa)
                update = True
            
            else: update = True
            
            if update :
                old_q = Q[(x,y)][a]

                if first_visit_MC:
                  #way 1
                  returns[sa].append(G)
                  Q[(x,y)][a] = np.mean(returns[sa])

                else:
                  #way 2
                  #Q[(x,y)][a] = old_q + LEARNING_RATE * (G - old_q)
                  returns[sa].append(G)
                  Q[(x,y)][a] = np.mean(returns[sa])


        #update policy
        for s in policy.keys():
            policy[s] = max_dict(Q[s])[0]

    #find V
    V = {}
#    for i in range(ginDim):
#      for j in range(ginDim):
#            V[(i,j)] = []
   
    for s, Qs in Q.items():
        #result way #1
        V[s] = max_dict(Q[s])[1]
        #result way #2
        x,y = s
        state_action_values[x][y] = np.array(list(v for a,v in Q[s].items())) 

    state_values = np.max(state_action_values[:,:,:], axis=-1)
    best_actions = np.argmax(state_action_values[:,:,:], axis=-1)

    return episodes, np.around(state_values,1), best_actions


In [2]:
import math
import matplotlib.pyplot as plt
from matplotlib.table import Table

def draw_policy(gdicPolicy, gsFigName):
    '''
    gdicPolicy: dictionary of values with cell coordinates as keys, and taken actions at the cells as values 
    (0, 0) [0 1 2 3]
    (0, 1) [0 1 2 3]
    ...

    gsFigName: output file name for visualization (e.g. gridworld_opt_policy_VI.png)
    '''

    # left, up, right, down
    ACTIONS = [np.array([0, -1]),
            np.array([-1, 0]),
            np.array([0, 1]),
            np.array([1, 0])]

    ACTIONS_FIGS=[ '←', '↑', '→', '↓']

    fig, ax = plt.subplots()
    ax.set_axis_off()
    tb = Table(ax, bbox=[0, 0, 1, 1])


    dic_policy = dict(np.ndenumerate(gdicPolicy))

    nrows, ncols = int(math.sqrt(len(dic_policy))), int(math.sqrt(len(dic_policy)))
    width, height = 1.0 / ncols, 1.0 / nrows

    # Add cells
    # for (i, j), val in np.ndenumerate(gvOptValues):
    for cell, pol in dic_policy.items():
        
        #val=''
        #for ba in pol:
        #    val+=ACTIONS_FIGS[int(ba)]
        
        val = ACTIONS_FIGS[int(pol)]

        i = cell[0]
        j = cell[1]
        
      
        tb.add_cell(i, j, width, height, text=val,
                loc='center', facecolor='white')

    # Row and column labels...
    for i in range(int(math.sqrt(len(dic_policy)))):
        tb.add_cell(i, -1, width, height, text=i+1, loc='right',
                    edgecolor='none', facecolor='none')
        tb.add_cell(-1, i, width, height/2, text=i+1, loc='center',
                   edgecolor='none', facecolor='none')

    ax.add_table(tb)

    plt.savefig(gsFigName)
    plt.close()

In [15]:
iterCt , policy_values = monte_carlo_on_policy_prediction ([0.25 ,0.25 ,0.25 ,0.25], 
                                                           episodes =100000, ginDim =4 , 
                                                           first_visit_MC = True)

100%|██████████| 100000/100000 [34:27<00:00, 48.37it/s]


In [16]:
iterCt

100000

In [17]:
policy_values #[100000]

array([[-1.5, -1.5, -1.5, -1.5],
       [-1.3, -1.3, -1.3, -1.3],
       [-1. , -1. , -1. , -1. ],
       [-1.1, -1.1, -1.1, -1.2]])

In [14]:
policy_values #[10000]

array([[-1.5, -1.5, -1.5, -1.5],
       [-1.2, -1.2, -1.2, -1.2],
       [-0.8, -0.8, -0.8, -0.8],
       [-1.1, -1.1, -1.1, -1.2]])

In [10]:
policy_values #[1000]

array([[-1.1, -1.1, -1.1, -1.1],
       [-0.8, -0.8, -0.8, -0.8],
       [-1.2, -1.2, -1.2, -1.2],
       [-1.3, -1.3, -1.3, -1.3]])

In [9]:
policy_values #[1000]

array([[-1.1, -1.1, -1.1, -1.1],
       [-0.8, -0.8, -0.8, -0.8],
       [-1.2, -1.2, -1.2, -1.2],
       [-1.3, -1.3, -1.3, -1.3]])

In [6]:
policy_values #[100]

array([[-0.9, -1.1, -0.9, -0.9],
       [-0.6, -0.6, -0.6, -0.6],
       [-0.9, -0.9, -0.9, -0.9],
       [-1.4, -1.3, -1.3, -1.4]])

In [ ]:
policy_values #[10]

array([[-2.2, -0.4, -2.1, -2.3],
       [-1.6, -1.6, -1.6, -1.6],
       [-0.7, -0.7, -0.7, -0.7],
       [-0.8, -0.8, -0.9, -0.8]])

In [ ]:
iterCt, policy_values = monte_carlo_on_policy_prediction ([0.25 ,0.25 ,0.25 ,0.25], 
                                                           episodes =10, ginDim =4 , 
                                                           first_visit_MC = False)

100%|██████████| 10/10 [00:00<00:00, 627.60it/s]


In [ ]:
iterCt

10

In [ ]:
policy_values

array([[-0.5, -0.5, -0.5, -0.5],
       [-0.3, -0.3, -0.3, -0.3],
       [-0.4, -0.4, -0.4, -0.4],
       [-0.6, -0.6, -0.6, -0.6]])

In [ ]:
iterCt, policy_values, policy = monte_carlo_ES([0.25 ,0.25 ,0.25 ,0.25], 
                                       episodes =10, ginDim =4 , 
                                       first_visit_MC = True)

100%|██████████| 10/10 [00:02<00:00,  4.82it/s]


In [ ]:
iterCt

10

In [ ]:
policy_values

array([[  0. ,   0. ,   0. ,   0. ],
       [  0. ,  -8.8, -10. ,   0. ],
       [  0. ,   0. ,   0. ,   0. ],
       [  0. ,   0. ,   0. ,   0. ]])

In [ ]:
policy

array([[0, 2, 3, 0],
       [3, 0, 3, 0],
       [2, 3, 1, 3],
       [1, 1, 0, 0]])

In [ ]:
s_fig_name = "policy_MC_monte_carlo_ES_first_visit_MC_True_v1.png"
draw_policy(policy, s_fig_name)

In [60]:
iterCt, policy_values, policy = monte_carlo_ES([0.25 ,0.25 ,0.25 ,0.25], 
                                       episodes =1000, ginDim =4 , 
                                       first_visit_MC = False)




  0%|          | 0/1000 [00:00<?, ?it/s]


  1%|          | 10/1000 [00:00<00:41, 23.68it/s]


  1%|          | 11/1000 [00:00<02:26,  6.75it/s]


  1%|          | 12/1000 [00:01<02:46,  5.93it/s]


  1%|▏         | 13/1000 [00:01<02:54,  5.67it/s]


  1%|▏         | 14/1000 [00:02<08:49,  1.86it/s]


  2%|▏         | 15/1000 [00:03<08:31,  1.93it/s]


  2%|▏         | 17/1000 [00:03<06:49,  2.40it/s]


  2%|▏         | 18/1000 [00:03<05:17,  3.10it/s]


  2%|▏         | 20/1000 [00:03<04:10,  3.91it/s]


  2%|▏         | 21/1000 [00:03<03:49,  4.26it/s]


  3%|▎         | 30/1000 [00:04<02:43,  5.93it/s]


  3%|▎         | 33/1000 [00:04<02:10,  7.42it/s]


  4%|▎         | 36/1000 [00:04<01:41,  9.54it/s]


  4%|▍         | 39/1000 [00:04<01:20, 11.94it/s]


  4%|▍         | 42/1000 [00:04<01:14, 12.90it/s]


  5%|▌         | 53/1000 [00:04<00:55, 17.17it/s]


  6%|▌         | 57/1000 [00:07<04:23,  3.58it/s]


  6%|▌         | 60/1000 [00:08<03:32,  4.42it/s]


  6%|▋         | 6

In [62]:
iterCt

1000

In [61]:
policy_values #[1000]

array([[ 0. , -8.5, -8.8, -9.8],
       [-1.6, -9.5, -9.8, -9.5],
       [-8.8, -9.7, -9.3, -8.5],
       [-9.8, -9.4, -8.6,  0. ]])

In [59]:
policy_values #[100]

array([[ 0. , -5.9,  0. , -9.6],
       [-8.8, -9.2, -8.9, -6.7],
       [-9.8, -9.8, -9.4,  0. ],
       [-9.3,  0. , -9.2,  0. ]])

In [56]:
policy_values #[50]

array([[  0. ,  -5.1,  -9.9,   0. ],
       [  0. ,  -9.3, -10. , -10. ],
       [ -9.8,  -9.9,  -9.7,  -9. ],
       [  0. , -10. ,  -9.3,   0. ]])

In [53]:
policy_values #[10]

array([[ 0. ,  0. ,  0. ,  0. ],
       [ 0. , -9.6, -9.8, -9.8],
       [ 0. ,  0. , -5.1,  0. ],
       [ 0. ,  0. ,  0. ,  0. ]])

In [22]:
policy_values #[1]

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [ ]:
policy

array([[0, 2, 3, 3],
       [2, 3, 2, 0],
       [2, 0, 1, 0],
       [1, 1, 0, 0]])

In [ ]:
s_fig_name = "policy_MC_monte_carlo_ES_first_visit_MC_False_v1.png"
draw_policy(policy, s_fig_name)